In [6]:
from resemblyzer import preprocess_wav, VoiceEncoder
from spectralcluster import SpectralClusterer
from pathlib import Path
import librosa
import numpy as np
from scipy.io.wavfile import write
from scipy.io import wavfile
import os
import speech_recognition as sr
import time
import azure.cognitiveservices.speech as speechsdk
import subprocess

In [7]:
audio_file_path = 'file.wav'

In [8]:
wav_fpath = Path(audio_file_path)

In [9]:
wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder('cpu')
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)
clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    # p_percentile=0.90,
    # gaussian_blur_sigma=1
    )
labels = clusterer.predict(cont_embeds)
def create_labelling(labels,wav_splits):
    sampling_rate = 16000
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0
    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))
    return labelling
labelling = create_labelling(labels,wav_splits)



Loaded the voice encoder model on cpu in 0.02 seconds.
(720, 256)


In [10]:
labelling

[('1', 0, 8.36),
 ('0', 8.36, 19.1),
 ('1', 19.1, 20.78),
 ('0', 20.78, 26.36),
 ('1', 26.36, 26.6),
 ('0', 26.6, 41.9),
 ('1', 41.9, 43.94)]